In [4]:
from ase.build import molecule
from ase.calculators.emt import EMT
from ase.optimize import BFGS
from ase.vibrations import Vibrations
from ase.io.trajectory import Trajectory

import nglview as nv

import numpy as np
import os

from numpy.linalg import norm

In [5]:
### Computing first trajectory for further vizualisation
molecule_name = "CO2"
atoms = molecule(molecule_name, calculator=EMT())
folder="vibrations"

# Relax and get vibrational properties
opt = BFGS(atoms, logfile=None)
opt.run(fmax=0.001)

vibname =os.path.join(folder,molecule_name)
vib = Vibrations(atoms, name='test')
vib.run()

# Extract rotational motions
ndofs=3*len(atoms)
is_not_linear=int(not(len(atoms)==2 or atoms.get_angle(0,1,2)==0))
nrotations=ndofs-5-is_not_linear 
energies=np.absolute(vib.get_energies())

# Get the nrotations-largest energies, to eliminate translation and rotation energies
idx = np.argpartition(energies, -nrotations)[-nrotations:]
rot_energies=energies[idx]
for i in idx:
    vib.write_mode(n=i,nimages=60)

In [6]:
traj = Trajectory('test.0.traj')
positions=[list(x) for x in traj[0].get_positions()]
directions=vib.get_mode(n=0).T
directions[(directions>-1e-4)&(directions<1e-4)]=0.
directions=directions/norm(directions,axis=0)
directions=directions.T


view = nv.show_asetraj(traj,delay=10) #gui=True
view._js(f'''
var shape = new NGL.Shape("my_shape")
let tab=[];
let positions={positions};
for (let i=0; i<3;i++){{
  var arrowBuffer = new NGL.ArrowBuffer({{
  position1: new Float32Array(positions[i]),
  position2: new Float32Array(positions[i]),
  color: new Float32Array([ 0, 1, 0 ]),
  radius: new Float32Array([ 0.1 ]),
}})
shape.addBuffer(arrowBuffer)
globalThis.arrowBuffer = arrowBuffer;
tab.push(arrowBuffer);}}

globalThis.tab=tab;
var shapeComp = this.stage.addComponentFromObject(shape)


shapeComp.addRepresentation("buffer")
shapeComp.autoView()
''')

sinus=np.sin(np.arange(0,60)/15*np.pi/2)
def on_frame_change(change):
    frame = change['new']
    positions=traj[frame].get_positions()
    positions2=positions+directions*sinus[frame]*2
    positions=[list(x) for x in positions]
    positions2=[list(x) for x in positions2]
    view._js(f'''
    let positions={positions};
    let positions2={positions2};
    for(let i =0; i<3; i++){{
      globalThis.tab[i].setAttributes({{
        position1: new Float32Array(positions[i]),
        position2: new Float32Array(positions2[i])
      }})}}
    
    this.stage.viewer.requestRender()
    ''')
view.observe(on_frame_change, names=['frame'])
view.parameters = dict(
        clipDist=-100
    )
view.player.parameters=dict(delay=50)
view

NGLWidget(max_frame=59)